Set up the database.

In [1]:
!createdb wells

createdb: database creation failed: ERROR:  database "wells" already exists


In [1]:
!psql -c "CREATE EXTENSION postgis;" wells

CREATE EXTENSION


In [1]:
%load_ext sql

In [2]:
%%sql
postgresql://postgres:@localhost/wells

'Connected: postgres@wells'

In [5]:
%%sql
DROP TABLE IF EXISTS states CASCADE;
CREATE TABLE states 
(
    id serial,
    state char(2) UNIQUE,
    name VARCHAR,
    source_url VARCHAR,
    description TEXT,
    last_updated TIMESTAMPTZ,
    api_prefix CHAR(2),
    PRIMARY KEY (id)
);
CREATE INDEX idx_state_id ON states(id);
CREATE INDEX idx_state_st ON states(state);

Done.
Done.
Done.
Done.


[]

Injection Wells: (https://www.epa.gov/uic/class-ii-oil-and-gas-related-injection-wells)
Well types and statuses: (http://www.ogsrlibrary.com/definitions_well_status_oil_gas_ontario)
INJECTION = FRACKING

In [4]:
%%sql
DROP TYPE IF EXISTS enum_well_type CASCADE;
CREATE TYPE enum_well_type AS ENUM
(
    'OIL',
    'GAS',
    'OILANDGAS',
    'INJECTION',
    'STORAGE',
    'DISPOSAL',
    'SERVICE',
    'DRY HOLE',
    'OBSERVATION',
    'TEST',
    'WATER',
    'OTHER',
    'UNKNOWN'
);
DROP TYPE IF EXISTS enum_well_status CASCADE;
CREATE TYPE enum_well_status AS ENUM
(
    'PERMITTED',
    'CANCELLED',
    'ACTIVE',
    'INACTIVE',
    'A', -- ABANDONED
    'TA',
    'SI', -- SHUT IN
    'PA',
    'DRY',
    'ORPHAN',
    'UNKNOWN'
);
DROP TYPE IF EXISTS enum_well_category CASCADE;
CREATE TYPE enum_well_category AS ENUM
(
    'CONVENTIONAL',
    'UNCONVENTIONAL',
    'FRAC',
    'CBM',
    'SERVICE',
    'TEST',
    'STORAGE',
    'WATER',
    'OTHER',
    'UNKNOWN'
);


DROP TABLE IF EXISTS wells;
CREATE TABLE wells
(
    id SERIAL PRIMARY KEY,
    api varchar NOT NULL CONSTRAINT plain_api CHECK (api ~ '^[A-Z0-9]+$'),
    state char(2) REFERENCES states(state) NOT NULL,
    status enum_well_status NOT NULL,
    type enum_well_type,
    category enum_well_category,
    date DATE,
    description TEXT,
    location GEOGRAPHY(POINT, 4326),
    capture_time TIMESTAMPTZ,
    UNIQUE (api, status, date)
);

CREATE INDEX idx_well_id ON wells(id);
CREATE INDEX idx_api ON wells(api);
CREATE INDEX idx_api_status_date ON wells(api, status, date);
CREATE INDEX idx_api_capture_time ON wells(api, capture_time);
CREATE INDEX idx_well_location ON wells(location);
CREATE INDEX idx_well_status ON wells(status);
CREATE INDEX idx_well_type ON wells(type);

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [7]:
import psycopg2

conn = psycopg2.connect(database="wells")
with conn:
    with conn.cursor() as cur:
        with open('states-data.csv', 'r') as f:
            cur.copy_expert('COPY states(api_prefix, state, name) FROM STDIN WITH DELIMITER \',\' CSV HEADER', f)
            #, 'states', sep=',', columns='api_prefix, state, name'))
conn.close()            

In [1]:
!shp2pgsql -s 4326 -d shapefiles/cb_2016_us_state_5m/cb_2016_us_state_5m.shp states_shape | psql -d wells

Field aland is an FTDouble with width 14 and precision 0
Field awater is an FTDouble with width 14 and precision 0
Shapefile type: Polygon
Postgis type: MULTIPOLYGON[2]
SET
SET
ERROR:  column not found in geometry_columns table
CONTEXT:  PL/pgSQL function dropgeometrycolumn(character varying,character varying,character varying,character varying) line 34 at RAISE
SQL statement "SELECT public.DropGeometryColumn('',$1,$2,$3)"
PL/pgSQL function dropgeometrycolumn(character varying,character varying,character varying) line 5 at SQL statement
NOTICE:  table "states_shape" does not exist, skipping
DROP TABLE
BEGIN
CREATE TABLE
ALTER TABLE
                      addgeometrycolumn                       
--------------------------------------------------------------
 public.states_shape.geom SRID:4326 TYPE:MULTIPOLYGON DIMS:2 
(1 row)

INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 1
INSERT 0 